In [1]:
import os, shutil
import random
import tensorflow as tf
from tensorflow.keras import (models, layers, optimizers)

import plotly.express as ex
import plotly.graph_objects as go

2023-01-24 20:55:26.924081: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 20:55:27.188897: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 20:55:27.232775: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-24 20:55:27.232795: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
data_path = "../data/raw"
x_train = os.listdir(data_path + "/train")
# x_test = os.listdir(data_path + "/test1")
x_train = [os.path.join(data_path, "train", x) for x in x_train]
# x_test = [os.path.join(data_path, "test1", x) for x in x_test]
print(len(x_train))
# print(len(x_test))
print(x_train[:5])
# print(x_test[:5])

25000
['../data/raw/train/dog.11082.jpg', '../data/raw/train/cat.2955.jpg', '../data/raw/train/cat.12060.jpg', '../data/raw/train/dog.971.jpg', '../data/raw/train/cat.6966.jpg']


In [3]:
x_train = random.choices(x_train, k=3000)
# x_test = random.choices(x_train, k=250)
print(len(x_train))
# print(len(x_test))

3000


In [4]:
data_path = "../data/processed"
if os.path.exists(data_path):
    shutil.rmtree(data_path)
os.makedirs(data_path + "/train/dogs")
os.makedirs(data_path + "/train/cats")

In [5]:
for x in x_train:
    if "dog" in x:
        shutil.copy(x, data_path + "/train/dogs")
    else:
        shutil.copy(x, data_path + "/train/cats")

In [6]:
print(len(os.listdir(data_path + "/train/dogs")))
print(len(os.listdir(data_path + "/train/cats")))

1424
1382


In [22]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 17, 17, 128)     

In [23]:
model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.RMSprop(learning_rate=1e-4),
    metrics=['acc']
)

In [24]:
batch_size = 16
x_train, x_valid = tf.keras.utils.image_dataset_from_directory(
    (data_path + "/train"),
    batch_size=batch_size,
    image_size=(150, 150),
    validation_split=.2,
    subset="both",
    seed=42
)

Found 2806 files belonging to 2 classes.
Using 2245 files for training.
Using 561 files for validation.


In [25]:
x_train_count = len(x_train) * batch_size
x_valid_count = len(x_valid) * batch_size
print(x_train_count)
print(x_valid_count)

2256
576


In [26]:
history = model.fit(
    x_train.repeat(),
    steps_per_epoch=int(x_train_count/batch_size),
    epochs=25,
    validation_data=x_valid.repeat(),
    validation_steps=int(x_valid_count/batch_size)
)

Epoch 1/25
141/141 [==============================] - 21s 142ms/step - loss: 2.2590 - acc: 0.5287 - val_loss: 0.7132 - val_acc: 0.5383
Epoch 2/25
  3/141 [..............................] - ETA: 16s - loss: 0.8536 - acc: 0.4792

KeyboardInterrupt: 

In [ ]:
# save model
model.save("./model.h5")

In [ ]:
# load model
# model = models.load_model("./model.h5")

In [ ]:
acc = history.history['acc']
vall_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = [x for x in range(1, len(acc) + 1)]

fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=acc, name="Training Accuracy"))
fig.add_trace(go.Scatter(x=epochs, y=vall_acc, name="Validation Accuracy"))
fig.update_layout(
    title="Training and Validation Accuracy",
    xaxis_title="Epochs",
    yaxis_title="Accuracy"
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=loss, name="Training Loss"))
fig.add_trace(go.Scatter(x=epochs, y=val_loss, name="Validation Loss"))
fig.update_layout(
    title="Training and Validation Loss",
    xaxis_title="Epochs",
    yaxis_title="Loss"
)
fig.show()